# Data quality: rimozione duplicati
### Elimino i documenti che compaiono più di una volta, poi li reinserisco esattamente una volta

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import time
import sys
from pymongo import MongoClient
import nltk
from difflib import SequenceMatcher
import random

#Connessione al DB
client = MongoClient(host='localhost', port=27018)
db = client.kiva_DB 

In [3]:
def get_economic_indicator(indicator,country, year, indicators):
    
   
    df = indicators[indicator]
    if(country in df.index and str(year) in df.columns):
        return df.loc[country].to_dict()[str(year)]
    else:
        return np.nan
    
def get_string_of_IDs(nums):
    IDs = []
    for i in nums:
        IDs.append(str(i))
    return ','.join(IDs)

url = 'https://api.kivaws.org/v1/loans/search.json'
params = {
        'per_page' : 100,
        'sort_by' : 'oldest',
        'ids_only': 'true'
            }
folder = 'C:/Users/pablo/Desktop/DS - Primo anno Primo semestre/Data_Management/Progetto/{}.csv'
indicators = {'GDP' : pd.read_csv(folder.format('GDP')).set_index('Unnamed: 0'),
                 'GNI': pd.read_csv(folder.format('GNI')).set_index('Unnamed: 0'),
                 'GNI_atlas': pd.read_csv(folder.format('GNI_atlas')).set_index('Unnamed: 0'),
                 'GNI_pc': pd.read_csv(folder.format('GNI_pc')).set_index('Unnamed: 0')}

In [10]:
def delete_duplicates(duplicates_ids):
#b) Cancella tutti i documenti duplicati
    for ID in duplicates_ids:
        db.loans.delete_many({'id':ID})
    print('',len(duplicates_ids), ' loans deleted from database.')
        
        
def reinsert_deleted(duplicates_ids):
#c) reimmetti i documenti eliminati (ma singoli, senza i duplicati)
    strings_of_IDs = []
    for i in range(0,len(duplicates_ids), 100):
        strings_of_IDs.append(get_string_of_IDs(duplicates_ids[i:i+100]))
    #Creo una lista di stringhe, ogni stringa contiene 100 ID (o meno, se è l'ultima) per la richiesta, così iterando su questa
    #lista, effettuo direttamente le richieste
    to_insert = []
    for ID in strings_of_IDs:
        loans = requests.get('https://api.kivaws.org/v1/loans/{}.json'.format(ID)).json()
        for loan in loans['loans']:
            loan['location']['GDP']= get_economic_indicator('GDP',loan['location']['country'], loan['posted_date'][0:4], indicators)
            loan['location']['GNI']= get_economic_indicator('GNI',loan['location']['country'], loan['posted_date'][0:4], indicators)
            loan['location']['GNI_atlas']= get_economic_indicator('GNI_atlas',loan['location']['country'], loan['posted_date'][0:4], indicators)
            loan['location']['GNI_pc']= get_economic_indicator('GNI_pc',loan['location']['country'], loan['posted_date'][0:4],indicators)
            to_insert.append(loan)
    if(len(to_insert)==1):
        db.loans.insert_one(to_insert[0])
    else:
        db.loans.insert_many(to_insert)
    print('Loans with ID ', duplicates_ids, ' inserted into database.')
    

#a) Query per trovare gli indici dei duplicati
t = time.time()
pipeline = [
    {"$group": {"_id": "$id", "count": {"$sum" : 1}}},
    {'$match':{"count": {"$gt":1}}}
    ]

dupls = db.loans.aggregate(pipeline = pipeline, allowDiskUse=True)
print('Time elapsed to complete query: ', time.time() - t)
duplicates = {}
for dp in dupls:
    duplicates[dp['_id']] = dp['count']

dupls.close()
count = db.loans.count()
duplicates_ids = list(duplicates.keys())
if(len(duplicates_ids) > 0):
    print('', len(duplicates_ids), ' duplicate IDs found. Engaging deletion protocol.')
    delete_duplicates(duplicates_ids)
    if(count - np.sum(list(duplicates.values())) == db.loans.count()):
        print('Deletion protocol successfull, hostiles eliminated. Beginning replacement.')
    reinsert_deleted(duplicates_ids)
else:
    print('No duplicates found, I repeat, no duplicates found, Python 5 returns to base, over.')


Time elapsed to complete query:  11.414714574813843
No duplicates found, I repeat, no duplicates found, Python 5 returns to base, over.


# Per ogni nome di paese nel DB che non corrisponde ad alcun paese nell'archivio della World Bank, cerco in quest'ultimo il nome più simile e inserisco i valori degli indicatori nel DB

In [13]:

df_GDP = pd.melt(indicators['GDP'].reset_index(), id_vars=['Unnamed: 0']).sort_values(['Unnamed: 0','variable']).rename(columns = {'Unnamed: 0':'Country', 'variable':'Year', 'value':'GDP'}).reset_index(drop=True)
df_GDP.set_index(['Country','Year'],inplace=True)
df_GNI = pd.melt(indicators['GNI'].reset_index(), id_vars=['Unnamed: 0']).sort_values(['Unnamed: 0','variable']).rename(columns = {'Unnamed: 0':'Country', 'variable':'Year', 'value':'GNI'}).reset_index(drop=True)
df_GNI.set_index(['Country','Year'],inplace=True)
df_GNI_atlas = pd.melt(indicators['GNI_atlas'].reset_index(), id_vars=['Unnamed: 0']).sort_values(['Unnamed: 0','variable']).rename(columns = {'Unnamed: 0':'Country', 'variable':'Year', 'value':'GNI_atlas'}).reset_index(drop=True)
df_GNI_atlas.set_index(['Country','Year'],inplace=True)
df_GNI_pc = pd.melt(indicators['GNI_pc'].reset_index(), id_vars=['Unnamed: 0']).sort_values(['Unnamed: 0','variable']).rename(columns = {'Unnamed: 0':'Country', 'variable':'Year', 'value':'GNI_pc'}).reset_index(drop=True)
df_GNI_pc.set_index(['Country','Year'],inplace=True)

gdp_countries = list(df_GDP.Country.unique())
gni_countries = list(df_GNI.Country.unique())
gni_atlas_countries = list(df_GNI_atlas.Country.unique())
gni_pc_countries = list(df_GNI_pc.Country.unique())

kiva_countries = db.loans.distinct('location.country')

In [5]:
def match_by_probability(k_c,wb_c):
    kiva_country = k_c.strip(' ').lower()
    wb_country = wb_c.strip(' ').lower()
    probs = [SequenceMatcher(None, kiva_country, wb_country).ratio()]
    for word in kiva_country.split(' '):
        probs.append(SequenceMatcher(None, word.strip(' '), wb_country).ratio())
    for word in wb_country.split(' '):
        probs.append(SequenceMatcher(None, kiva_country, word.strip(' ')).ratio())
    return max(probs)
    
     
# def match_by_distance(k_c,wb_c):
#     kiva_country = k_c.strip(' ').lower()
#     wb_country = wb_c.strip(' ').lower()
#     dist = [nltk.edit_distance(kiva_country, wb_country)]
#     for word in kiva_country:
#         dist.append(nltk.edit_distance(word.strip(' '), wb_country))
#     for word in wb_country:
#         dist.append(nltk.edit_distance(kiva_country, word.strip(' ')))
#     return min(dist)


def match_country(kiva_country,wb_countries):
#     min_dist = 100
    max_prob= 0
    max_prob_wb_country = None
#     min_dist_wb_country = None

    for wb_country in wb_countries:
        prob = match_by_probability(kiva_country, wb_country)
        if(prob > max_prob):
            max_prob=prob
            max_prob_wb_country=wb_country
            
#         dist = match_by_distance(kiva_country, wb_country)
#         if(dist < min_dist):
#             min_dist = dist
#             min_dist_wb_country = wb_country
    return max_prob_wb_country
        


matches = {}

for kiva_country in kiva_countries:
    if(kiva_country not in gni_countries):
        matches[kiva_country] = match_country(kiva_country,gni_countries)
    
matches['Congo (DRC)']='Congo, Dem. Rep.'
matches['Palestine']= 'West Bank and Gaza'



In [31]:
def get_list_of_loans(cursor):
    return [x for x in cursor]

def modify_loans(loans_to_modify, c):
    try: 
        country = matches[c]
    except KeyError:
        print('No such country in map')
        return

    for l in loans_to_modify:
        loan = l
        try: 
            GDP = df_GDP.loc[country,loan['posted_date'][0:4]]
        except KeyError:
            GDP=np.nan
            print('No GDP for', country, 'in the', loan['posted_date'][0:4])
            
        try: 
            GNI = df_GNI.loc[country,loan['posted_date'][0:4]]
        except KeyError:
            GNI=np.nan
            print('No GNI for', country, 'in the', loan['posted_date'][0:4])
            
        try: 
            GNI_atlas = df_GNI_atlas.loc[country,loan['posted_date'][0:4]]
        except KeyError:
            GNI_atlas=np.nan
            print('No GNI_atlas for', country, 'in the', loan['posted_date'][0:4])
            
        try: 
            GNI_pc = df_GNI_pc.loc[country,loan['posted_date'][0:4]]
        except KeyError:
            GNI_pc=np.nan
            print('No GNI_pc for', country, 'in the', loan['posted_date'][0:4])
        
        
        loan['location']['GDP'] = str(GDP)
        loan['location']['GNI'] = str(GNI)
        loan['location']['GNI_atlas'] = str(GNI_atlas)
        loan['location']['GNI_pc'] = str(GNI_pc)
        
        db.loans.delete_one({'id':loan['id']})
        db.loans.insert_one(loan)

for country in matches.keys():
    loans_to_modify = get_list_of_loans(db.loans.find({'location.country':country}))
    modify_loans(loans_to_modify, country)


No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep

No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep

No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep

No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep

No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep. in the 2017
No GDP for Congo, Dem. Rep. in the 2017
No GNI for Congo, Dem. Rep. in the 2017
No GNI_atlas for Congo, Dem. Rep. in the 2017
No GNI_pc for Congo, Dem. Rep

...

No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and 

No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and 

No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and 

No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and Gaza in the 2018
No GNI_pc for West Bank and Gaza in the 2018
No GDP for West Bank and Gaza in the 2018
No GNI for West Bank and Gaza in the 2018
No GNI_atlas for West Bank and 

KeyboardInterrupt: 

In [29]:
#sum([len(loans_to_modify[key]) for key in loans_to_modify.keys()])
loans_to_modify[0]['posted_date']

'2014-05-26T07:20:01Z'

In [19]:
df_GDP.loc['Afghanistan','2005']

GDP    6,275,073,571.54659
Name: (Afghanistan, 2005), dtype: object

In [116]:
exit()